# Coursera_capstone_project
## Week 3 part3

In [1]:
import pandas as pd
import numpy  as np
import json # library to handle JSON files
#!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#We upload the first dataset where we save it as csv file in part 2
my_df=pd.read_csv('Capstone_part2.csv')
my_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


### Explore the data

In [3]:
neighborhoods = my_df[['Borough','Neighborhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 15 boroughs and 103 neighborhoods.


In [4]:
neighborhoods['Borough'].value_counts()

North York                24
Scarborough               17
Downtown Toronto          17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East York                  4
East Toronto               4
Downtown Toronto Stn A     1
Mississauga                1
Etobicoke Northwest        1
Queen's Park               1
East Toronto Business      1
East York/East Toronto     1
Name: Borough, dtype: int64

In [5]:
# Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer") #why it;s to_explorer?
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [6]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
#Chose another object North York to analysis
North_York_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.75245,-79.32991
1,North York,Victoria Village,43.73057,-79.31306
2,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
3,North York,Don Mills North,43.74923,-79.36186
4,North York,Glencairn,43.70687,-79.44812


In [8]:
#Get the lat and lng for North York
address = 'North York, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [9]:
#Create map of Scarborough using latitude and longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=11.2)

# add markers to map
for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North York)  
    
map_North_York

#### Foursquare API

In [10]:
CLIENT_ID = 'CKHL1D1D1OFKBSLAUMC22VXCKQRQGH2GPHJAWXDJQS5DZLR5' # your Foursquare ID
CLIENT_SECRET = '4D2TPJQHRHCFJAZ5Q1X0S4WJR3BISU0NROOZOPGJGT11WRSN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
#get neighbor's name
North_York_data.loc[0, 'Neighborhood']

Your credentails:
CLIENT_ID: CKHL1D1D1OFKBSLAUMC22VXCKQRQGH2GPHJAWXDJQS5DZLR5
CLIENT_SECRET:4D2TPJQHRHCFJAZ5Q1X0S4WJR3BISU0NROOZOPGJGT11WRSN


'Parkwoods'

In [11]:
#Get neighbor's latitude and longtitude
neighborhood_latitude = North_York_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.75245000000007, -79.32990999999998.


#### Fetching API

In [12]:
#Creat url to get the top 100 venues that are in Parkwoods within a radius of 500 meters.
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CKHL1D1D1OFKBSLAUMC22VXCKQRQGH2GPHJAWXDJQS5DZLR5&client_secret=4D2TPJQHRHCFJAZ5Q1X0S4WJR3BISU0NROOZOPGJGT11WRSN&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100'

In [13]:
#Get results of url and define next step
results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [15]:
limit=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET requests
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            r['venue']['name'], 
            r['venue']['location']['lat'], 
            r['venue']['location']['lng'],  
            r['venue']['categories'][0]['name']) for r in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [16]:
North_York_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [17]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

There are 108 uniques categories.


In [18]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Automotive Shop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,...,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped.shape

(24, 109)

In [20]:
#Step_16 print neighbor and it's top 5 venues
num_top_venues = 5

for hood in North_York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0               Lawyer   0.5
1     Business Service   0.5
2  American Restaurant   0.0
3   Mexican Restaurant   0.0
4             Platform   0.0


----Bayview Village----
                        venue  freq
0                       Trail   0.4
1                     Dog Run   0.2
2  Construction & Landscaping   0.2
3                        Park   0.2
4          Mexican Restaurant   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.09
2      Sandwich Place  0.09
3            Pharmacy  0.04
4            Boutique  0.04


----Don Mills North----
                 venue  freq
0          Coffee Shop  0.25
1                 Park  0.25
2          Gas Station  0.25
3         Burger Joint  0.25
4  American Restaurant  0.00


----Don Mills South----
             venue  freq
0    Grocery Store  0.14
1              Gym  0.14
2     Intersect

#### Analyze the mean of the top 10 venues in each  region

#### find the 10 most common venues for each region

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Lawyer,Business Service,Women's Store,Gas Station,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court
1,Bayview Village,Trail,Construction & Landscaping,Park,Dog Run,Women's Store,Furniture / Home Store,Discount Store,Electronics Store,Fast Food Restaurant,Food & Drink Shop
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Boutique,Indian Restaurant,Greek Restaurant,Juice Bar,Liquor Store,Fast Food Restaurant,Comfort Food Restaurant
3,Don Mills North,Burger Joint,Park,Coffee Shop,Gas Station,Furniture / Home Store,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop
4,Don Mills South,Smoke Shop,Coffee Shop,Bubble Tea Shop,Supermarket,Grocery Store,Gym,Intersection,Electronics Store,Fast Food Restaurant,Furniture / Home Store


# Clustering

In [23]:
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 1, 1, 0, 1, 3, 1, 1, 1, 1])

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

North_York_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.75245,-79.32991,2,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Furniture / Home Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Food Court
1,North York,Victoria Village,43.73057,-79.31306,1,Park,Grocery Store,German Restaurant,Women's Store,Frozen Yogurt Shop,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant
2,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,1,Clothing Store,Women's Store,Bookstore,Toy / Game Store,Men's Store,Cosmetics Shop,Restaurant,Furniture / Home Store,Greek Restaurant,Movie Theater
3,North York,Don Mills North,43.74923,-79.36186,0,Burger Joint,Park,Coffee Shop,Gas Station,Furniture / Home Store,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop
4,North York,Glencairn,43.70687,-79.44812,1,Pizza Place,Grocery Store,Italian Restaurant,Fast Food Restaurant,Gas Station,Latin American Restaurant,Mediterranean Restaurant,Bank,Bakery,Japanese Restaurant


In [25]:
#This role is for when you need to use the cluster for data, but color=rainbow[cluster-1] can't take float, then you have to convert the Cluster Labels into int.
North_York_merged= North_York_merged.fillna(0)
North_York_merged[['Cluster Labels']] = North_York_merged[['Cluster Labels']].astype("int")

# Data visualization

In [26]:
#Last step, view the clustered data
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighborhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster examination
#### Cluster 1

In [27]:
North_York_merged.loc[(North_York_merged['Cluster Labels'] == 0), North_York_merged.columns[[2] + list(range(5, North_York_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,43.74923,Burger Joint,Park,Coffee Shop,Gas Station,Furniture / Home Store,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop
19,43.73367,Coffee Shop,Park,Nightclub,Women's Store,Department Store,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop
22,43.74778,Convenience Store,Park,Coffee Shop,Speakeasy,Furniture / Home Store,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop


#### Cluster 2

In [28]:
scarborough_merged.loc[scarborough_merNorth_York_merged.loc[(North_York_merged['Cluster Labels'] == 1), North_York_merged.columns[[2] + list(range(5, North_York_merged.shape[1]))]]ged['Cluster Labels'] == 1, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.73057,Park,Grocery Store,German Restaurant,Women's Store,Frozen Yogurt Shop,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant
2,43.72327,Clothing Store,Women's Store,Bookstore,Toy / Game Store,Men's Store,Cosmetics Shop,Restaurant,Furniture / Home Store,Greek Restaurant,Movie Theater
4,43.70687,Pizza Place,Grocery Store,Italian Restaurant,Fast Food Restaurant,Gas Station,Latin American Restaurant,Mediterranean Restaurant,Bank,Bakery,Japanese Restaurant
5,43.72168,Smoke Shop,Coffee Shop,Bubble Tea Shop,Supermarket,Grocery Store,Gym,Intersection,Electronics Store,Fast Food Restaurant,Furniture / Home Store
8,43.78097,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Coffee Shop,Restaurant,Bank,Juice Bar,Food Court,Women's Store,Supplement Shop
9,43.76476,Furniture / Home Store,Miscellaneous Shop,Sushi Restaurant,Caribbean Restaurant,Metro Station,Frame Store,Coffee Shop,Massage Studio,Bar,Fast Food Restaurant
10,43.78112,Trail,Construction & Landscaping,Park,Dog Run,Women's Store,Furniture / Home Store,Discount Store,Electronics Store,Fast Food Restaurant,Food & Drink Shop
11,43.73384,Turkish Restaurant,Italian Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Park,Bakery,Vietnamese Restaurant,Hockey Arena
13,43.72071,Vietnamese Restaurant,Hockey Arena,Department Store,Convenience Store,Fast Food Restaurant,Coffee Shop,Discount Store,Beer Store,Pizza Place,Hotel
14,43.71381,Park,Bakery,Basketball Court,Women's Store,Gas Station,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop


#### Cluster 3

In [29]:
North_York_merged.loc[(North_York_merged['Cluster Labels'] == 2), North_York_merged.columns[[2] + list(range(5, North_York_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.75245,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Furniture / Home Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Food Court
6,43.80225,Park,Residential Building (Apartment / Condo),Women's Store,Cosmetics Shop,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop
12,43.75698,Park,Women's Store,Furniture / Home Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court


#### Cluster 4

In [30]:
North_York_merged.loc[(North_York_merged['Cluster Labels'] == 3), North_York_merged.columns[[2] + list(range(5, North_York_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,43.73224,Construction & Landscaping,Women's Store,Gas Station,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,Frame Store


#### Cluster 5

In [31]:
North_York_merged.loc[(North_York_merged['Cluster Labels'] == 4), North_York_merged.columns[[2] + list(range(5, North_York_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,43.75788,Lawyer,Business Service,Women's Store,Gas Station,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court
